In [1]:
%run ../utils/utils.ipynb
import json
import requests

In [2]:
def getOrderInfo(session, config):
    url='https://show.bilibili.com/api/ticket/project/get?version=134&id='+str(config['projectID'])+'&project_id='+str(config['projectID'])
    response = session.get(url)
    response.encoding='utf-8'
    return response.json()

In [3]:
def getConfig():
    config={}
    with open('./config.json', 'r',encoding='utf-8') as file:
        config = json.load(file)
    return (config)

In [4]:
def getTicketInfo(json_data, config):
    info={}
    info['changciID']=json_data['data']['screen_list'][config['changciID']-1]['id']
    info['jiageID']=json_data['data']['screen_list'][config['changciID']-1]["ticket_list"][config["jiageID"] - 1]["id"]
    info['price']=json_data["data"]["screen_list"][config["changciID"]-1]["ticket_list"][config["jiageID"]-1]["price"]
    return info
def getBuyers(session, config):
    url='https://show.bilibili.com/api/ticket/buyer/list?is_default&projectId='+str(config['projectID'])
    response = session.get(url)
    response.encoding='utf-8'
    buyers=[]
    json_data = response.json()
    for i in range(0, config['count']):
        info=json_data['data']['list'][i]
        info["isBuyerInfoVerified"] = True
        info["isBuyerValid"] = True
        buyers.append(info)
    return buyers

In [5]:
def getToken(session, config, ticketInfo):
    # 获取token
    url = 'https://show.bilibili.com/api/ticket/order/prepare?project_id=' + str(config['projectID'])
    payload={
        'count':config['count'],
        'order_type':1,
        'project_id':config['projectID'],
        'screen_id':ticketInfo['changciID'],
        'sku_id':ticketInfo['jiageID'],
        'token':''
    }
    response = session.request("POST",
                               url,
                               data=payload)
    response.encoding='utf-8'
    data = response.json()
    return data['data']['token']

In [6]:
def createOrder(session, ticketInfo, buyers, config, token):
    url = "https://show.bilibili.com/api/ticket/order/createV2?project_id=" + str(config["projectID"])
    payload = {
        "buyer_info": buyers,
        "count": config["count"],
        "deviceId": "",
        "order_type": 1,
        "pay_money": ticketInfo["price"] * config["count"],
        "project_id": config["projectID"],
        "screen_id": ticketInfo["changciID"],
        "sku_id": ticketInfo["jiageID"],
        "timestamp": int(round(time.time() * 1000)),
        "token": token,
        "buyer":config['buyer'],
        "tel":config['tel']
    }
    response = session.request("POST",
                               url,
                               data=payload)
    data = response.json()
    print(data)
    if data["errno"] == 0:
        print("已成功抢到票, 请尽快支付 https://show.bilibili.com/orderlist")
        exit(0)
    elif data["errno"] == 100050:    # Token过期
        print("Token已过期! 正在重新获取")
        token=getToken(session, config, ticketInfo)
        payload['token'] = token
        response = session.request("POST",
                                   url,
                                   data=payload)

In [7]:
def flow():
    browser=create_selenium('https://www.bilibili.com/')
    time.sleep(2)
    cookies=login(browser)
    session = get_session(cookies)
    csrf = get_csrf(session)
    config=getConfig()
    json_data=getOrderInfo(session, config)
    ticketInfo=getTicketInfo(json_data, config)
    buyers=getBuyers(session, config)
    token=getToken(session, config, ticketInfo)
    createOrder(session, ticketInfo, buyers, config, token)

In [8]:
# flow()

In [ ]:
browser=create_selenium('https://www.bilibili.com/')
time.sleep(2)
cookies=login(browser)
session = get_session(cookies)
csrf = get_csrf(session)

In [ ]:
config=getConfig()
json_data=getOrderInfo(session, config)
ticketInfo=getTicketInfo(json_data, config)
buyers=getBuyers(session, config)
token=getToken(session, config, ticketInfo)
createOrder(session, ticketInfo, buyers, config, token)